In [2]:
import Dathoven

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.metrics import Precision
from keras.metrics import SparseTopKCategoricalAccuracy

In [4]:
SEQ_LEN = 30
BATCH_SIZE = 64
HIDDEN_UNITS = 50
EPOCHS = 50
VOCABULARY_SIZE = 201

In [5]:
print('Build model...')
model_interval = Sequential()
model_interval.add(Embedding(VOCABULARY_SIZE, HIDDEN_UNITS, input_length=SEQ_LEN, mask_zero=True))

# LSTM part
model_interval.add(LSTM(HIDDEN_UNITS, return_sequences=True))
model_interval.add(LSTM(HIDDEN_UNITS))

# Project back to vocabulary
model_interval.add(Dense(VOCABULARY_SIZE, activation='softmax'))
#model_interval.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])#metrics=[Precision(top_k=10)])
#model_interval.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[Precision(top_k=10)])
model_interval.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=[SparseTopKCategoricalAccuracy(k=10, name='sparse_top_k_categorical_accuracy')])


model_interval.summary()

Build model...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            10050     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 50)            20200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 201)               10251     
Total params: 60,701
Trainable params: 60,701
Non-trainable params: 0
_________________________________________________________________


In [6]:
interval_to_number = {}
number_to_interval = {}

for i in range(0,100):
    interval_to_number[i] = i
    interval_to_number[-i] = 100 + i
    number_to_interval[i] = i
    number_to_interval[100+i] = -i
    

In [7]:
model_interval.load_weights('models_checkpoints/weights-improvement-10-2.87.hdf5')

AttributeError: 'str' object has no attribute 'decode'

In [8]:
generated_vector = Dathoven.sample_seq([1], SEQ_LEN, model_interval, 50)
generated_intervals = [number_to_interval[number] for number in generated_vector]

In [9]:
from datetime import datetime 

In [10]:
Dathoven.from_intervals_to_midi(70, generated_intervals, datetime.now().strftime("%Y%m%d_%H%M%S")+'_generated_record_intervals_tuning.mid') 

We save the model to use with tensorflow js

In [11]:
model_interval.save("Model")